<a href="https://colab.research.google.com/github/Saumitra-Shukla/Regression-and-classification-of-tabular-data-/blob/master/Multiclass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
df=pd.read_csv('https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv',na_values=['NA','?'])
print(df[0:5])

   id job area   income  ...  pop_dense  retail_dense     crime  product
0   1  vv    c  50876.0  ...   0.885827      0.492126  0.071100        b
1   2  kd    c  60369.0  ...   0.874016      0.342520  0.400809        c
2   3  pe    c  55126.0  ...   0.944882      0.724409  0.207723        b
3   4  11    c  51690.0  ...   0.889764      0.444882  0.361216        b
4   5  kl    d  28347.0  ...   0.744094      0.661417  0.068033        a

[5 rows x 14 columns]


In [0]:

for i in df.columns:
  print(df[0:5][i])

In [46]:
dummies_j=pd.get_dummies(df['job'],prefix='job')
dummies_a=pd.get_dummies(df['area'],prefix='area')
df=pd.concat([df,dummies_j,dummies_a],axis=1)
df.columns.drop('job').drop('area').drop('id')

Index(['income', 'aspect', 'subscriptions', 'dist_healthy', 'save_rate',
       'dist_unhealthy', 'age', 'pop_dense', 'retail_dense', 'crime',
       'product', 'job_11', 'job_al', 'job_am', 'job_ax', 'job_bf', 'job_by',
       'job_cv', 'job_de', 'job_dz', 'job_e2', 'job_f8', 'job_gj', 'job_gv',
       'job_kd', 'job_ke', 'job_kl', 'job_kp', 'job_ks', 'job_kw', 'job_mm',
       'job_nb', 'job_nn', 'job_ob', 'job_pe', 'job_po', 'job_pq', 'job_pz',
       'job_qp', 'job_qw', 'job_rn', 'job_sa', 'job_vv', 'job_zz', 'area_a',
       'area_b', 'area_c', 'area_d'],
      dtype='object')

In [0]:
median=df['income'].median()
df['income']=df['income'].fillna(median)

In [0]:
from scipy.stats import zscore

df['income']=zscore(df['income'])
df['aspect']=zscore(df['aspect'])
df['subscriptions']=zscore(df['subscriptions'])
df['dist_healthy']=zscore(df['dist_healthy'])
df['save_rate']=zscore(df['save_rate'])
df['age']=zscore(df['age'])

In [0]:
x_columns=df.columns.drop('job').drop('area').drop('id').drop('product')
x=df[x_columns].values

dummies=pd.get_dummies(df['product'])
prod=dummies.columns
y=dummies.values

In [0]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [0]:
!pip install tf-nightly-gpu-2.0-preview

In [60]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


model=Sequential()
model.add(Dense(100,input_dim=x.shape[1],activation='relu',kernel_initializer='random_normal'))
model.add(Dense(50,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(25,activation='relu',kernel_initializer='random_normal'))
model.add(Dense(y.shape[1],activation='softmax',kernel_initializer='random_normal'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

monitor=EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=10,verbose=1,restore_best_weights=True)
checkpoint=ModelCheckpoint(filepath='best_weights.hdf5',monitor='val_loss', verbose=1, save_best_only=True)

#model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,callbacks=[monitor,checkpoint],verbose=2)

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpoint],verbose=2,epochs=1000)

Train on 1500 samples, validate on 500 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 1.13823, saving model to best_weights.hdf5
1500/1500 - 0s - loss: 1.5321 - accuracy: 0.3680 - val_loss: 1.1382 - val_accuracy: 0.5020
Epoch 2/1000

Epoch 00002: val_loss improved from 1.13823 to 1.08487, saving model to best_weights.hdf5
1500/1500 - 0s - loss: 1.1365 - accuracy: 0.4733 - val_loss: 1.0849 - val_accuracy: 0.5000
Epoch 3/1000

Epoch 00003: val_loss improved from 1.08487 to 1.02291, saving model to best_weights.hdf5
1500/1500 - 0s - loss: 1.0726 - accuracy: 0.5000 - val_loss: 1.0229 - val_accuracy: 0.5180
Epoch 4/1000

Epoch 00004: val_loss improved from 1.02291 to 0.90584, saving model to best_weights.hdf5
1500/1500 - 0s - loss: 0.9411 - accuracy: 0.6020 - val_loss: 0.9058 - val_accuracy: 0.6260
Epoch 5/1000

Epoch 00005: val_loss improved from 0.90584 to 0.82992, saving model to best_weights.hdf5
1500/1500 - 0s - loss: 0.8312 - accuracy: 0.6713 - val_loss: 0.8299 - val

In [51]:
print(x.shape,y.shape)

(2000, 47) (2000, 7)


In [61]:
!ls


best_weights.hdf5  sample_data


In [62]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp best_weights.hdf5 drive/'My Drive'/machine-learning/weights/multiclass_weights.hdf5